In [1]:
import nltk
import sys
import math
from decimal import *
import numpy as np

In [2]:
training_data = nltk.corpus.brown.tagged_sents(tagset='universal')[:10000]

In [3]:
tag_list = set()
word_set = set()
transition_dict = {}
tag_count = {}

In [4]:
for value in training_data:
    previous = "start"
    for data in value:
        word_set.add(data[0].lower())
        tag = data[1]
        tag_list.add(tag)

        if tag in tag_count:
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1

        if (previous, tag) in transition_dict:
            transition_dict[(previous, tag)] += 1
            previous = tag
        else:
            transition_dict[(previous, tag)] = 1
            previous = tag

In [6]:
word_map = {'start': 0}
i = 1
for word in word_set:
    if word in word_map:
        continue
    else:
        word_map[word] = i
        i += 1
word_map['undefined'] = len(word_map)

In [7]:
i = 1
tag_map = {'start': 0}
for tag in tag_list:
    if tag in tag_map:
        continue
    else:
        tag_map[tag] = i
        i += 1

In [8]:
count_dict = transition_dict
prob_dict = {}
for key in count_dict:
    den = 0
    val = key[0]
    for key_2 in count_dict:
        if key_2[0] == val:
            den += count_dict[key_2]
    prob_dict[key] = Decimal(count_dict[key])/(den)

In [9]:
transition_prob = prob_dict
for tag in tag_list:
    if ("start", tag) not in transition_prob:
        transition_prob[("start", tag)] = Decimal(
            1) / Decimal(len(word_set) + tag_count[tag])
for tag1 in tag_list:
    for tag2 in tag_list:
        if (tag1, tag2) not in transition_prob:
            transition_prob[(tag1, tag2)] = Decimal(
                1) / Decimal(len(word_set) + tag_count[tag1])

In [10]:

count_word = {}
for value in training_data:
    for data in value:
        word = data[0]
        tag = data[1]
        if (word, tag) in count_word:
            count_word[(word, tag)] += 1
        else:
            count_word[(word, tag)] = 1

In [11]:
word_count = count_word
emission_prob_dict = {}
for key in word_count:
    emission_prob_dict[key] = Decimal(
        word_count[key])/tag_count[key[1]]

In [12]:
transition_matrix = np.zeros((len(tag_map), len(tag_map)))
# print(transition_prob)
for key, value in transition_prob.items():
    i = tag_map[key[0]]
    j = tag_map[key[1]]
    transition_matrix[i, j] = value
# print(transition_matrix)

In [13]:
observation_matrix = np.zeros((len(tag_map), len(word_map)+1))

for key, value in emission_prob_dict.items():
    i = tag_map[key[1]]
    j = word_map[key[0].lower()]

    observation_matrix[i][j] = value
random_model = 1/len(tag_map)
for tag in tag_map.keys():
    i = tag_map[tag]
    j = word_map['undefined']

    observation_matrix[i][j] = random_model

In [14]:
def viterbi(obs, pi, a, b):
    print(obs)
    # obs.append(0)
    # obs_plain = []
    # for word in tagset[0]:
    #     obs_plain.append(word)
    #     obs.append(word_map[word[0].lower()])
    # print(obs_plain)
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]

    # init blank path
    path = np.zeros(T, dtype=int)
    # delta --> highest probability of any path that reaches state i
    delta = np.zeros((nStates, T))
    # phi --> argmax by time step for each state
    phi = np.zeros((nStates, T))

    # init delta and phi
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]]
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])

    path[T-1] = np.argmax(delta[:, T-1])
    for t in range(T-2, -1, -1):
        path[t] = phi[path[t+1], [t+1]]

    return path

In [15]:
pi = transition_matrix[0]
print(sum(pi))
a = transition_matrix
print(a[0:3])
b = observation_matrix
print(b[0:3])

1.0
[[0.00000000e+00 5.49000000e-02 1.20700000e-01 4.10000000e-02
  1.73000000e-02 6.50000000e-02 3.07000000e-02 5.00000000e-04
  2.40300000e-01 1.13400000e-01 1.96500000e-01 4.22000000e-02
  7.75000000e-02]
 [0.00000000e+00 4.53309157e-04 6.76941674e-02 1.60320338e-01
  1.94922937e-02 2.29676639e-02 2.35720762e-02 3.02206105e-04
  1.57147174e-01 5.25838622e-02 2.89815654e-01 1.17558175e-01
  8.80930795e-02]
 [0.00000000e+00 1.28875469e-03 1.89262832e-02 3.85153546e-02
  3.90308565e-02 1.13778629e-02 1.21879373e-02 3.68215627e-04
  4.43147507e-01 4.96722881e-02 2.87281832e-01 8.45791295e-02
  1.36239782e-02]]
[[0.         0.         0.         ... 0.         0.         0.07692308]
 [0.         0.         0.         ... 0.         0.         0.07692308]
 [0.         0.         0.         ... 0.         0.         0.07692308]]


In [16]:
def build_sentence(sentence):
    idx_sentence = [0]
    for wordtag in sentence:
        word = wordtag[0]
        try:
            idx = word_map[word]
        except:
            idx = word_map['undefined']
        idx_sentence.append(idx)
    return idx_sentence


def dec_sentence(states):
    final = []
    for state in states:
        for tag in tag_map.keys():
            if tag_map[tag] == state:
                final.append(tag)
    return final[1:]

In [17]:
test = nltk.corpus.brown.tagged_sents(tagset='universal')[10150:10153]

for t in test:
    print(t)
    idx_sentence = build_sentence(t)
    vt = viterbi(idx_sentence, pi, a, b)
    print(dec_sentence(vt))
    print()

[('Those', 'DET'), ('coming', 'VERB'), ('from', 'ADP'), ('other', 'ADJ'), ('denominations', 'NOUN'), ('will', 'VERB'), ('welcome', 'VERB'), ('the', 'DET'), ('opportunity', 'NOUN'), ('to', 'PRT'), ('become', 'VERB'), ('informed', 'VERB'), ('.', '.')]
[0, 21248, 10975, 8424, 1556, 21248, 10254, 18107, 17932, 20266, 12995, 3987, 6269, 15454]
['.', 'VERB', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'DET', 'NOUN', 'PRT', 'VERB', 'VERB', '.']

[('The', 'DET'), ('preparatory', 'ADJ'), ('class', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('introductory', 'ADJ'), ('face-to-face', 'ADJ'), ('group', 'NOUN'), ('in', 'ADP'), ('which', 'DET'), ('new', 'ADJ'), ('members', 'NOUN'), ('become', 'VERB'), ('acquainted', 'VERB'), ('with', 'ADP'), ('one', 'NUM'), ('another', 'DET'), ('.', '.')]
[0, 21248, 21248, 19584, 3974, 1515, 21248, 21248, 8717, 6164, 9207, 12845, 1975, 3987, 10053, 13650, 20195, 15420, 15454]
['ADP', 'DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'VE